# LangChain

In [1]:
!pip install langchain langchain_openai langchain_community langchainhub langgraph

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/23/98/c70fac0f1b3193ced86013b563119c27c68ac26b684815f407555224108d/langchain-0.1.0-py3-none-any.whl.metadata
  Using cached langchain-0.1.0-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for langchain_openai from https://files.pythonhosted.org/packages/f4/ba/6cfe8bdcc0c4f6b420d9f6128d1970273764eb1a7c2e3ef02efa2271dd30/langchain_openai-0.0.2-py3-none-any.whl.metadata
  Using cached langchain_openai-0.0.2-py3-none-any.whl.metadata (570 bytes)
  Obtaining dependency information for langchain_community from https://files.pythonhosted.org/packages/3b/db/ce4a9244a6a2d9fc07394f94fe4c7041a478cc14e03072f9e8bbece92667/langchain_community-0.0.10-py3-none-any.whl.metadata
  Using cached langchain_community-0.0.10-py3-none-any.whl.metadata (7.3 kB)
  Obtaining dependency information for langchainhub from htt


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os

# load api keys from openai_api_key.txt and tavily_api_key.txt files
with open(os.path.expanduser("openai_api_key.txt")) as f:
    openai_api_key = f.read().strip()

with open(os.path.expanduser("tavily_api_key.txt")) as f:
    tavily_api_key = f.read().strip()

# set api keys
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["TAVILY_API_KEY"] = tavily_api_key

In [4]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

# Construct the OpenAI Functions agent
agent = create_openai_functions_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [5]:
agent_executor.invoke({"input": "What is the weather in Rome?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Rome'}`


[{'url': 'https://hikersbay.com/weather/italy/rome?lang=en', 'content': '12℃ 53℉ 10℃ 49℉ Rome Weather forecast for Monday, January 8, 2024 Rome weather Today at 4:00 AM scattered clouds 9℃ 49℉  1021 72% 2.22 100% Rome Weather forecast for Wednesday, January 10, 2024  1010 78% 3 96% Rome Weather forecast for Tuesday, January 9, 2024 Rome weather Tomorrow at 1:00 AM overcast clouds  1021 77% 2.23 100% Rome Weather forecast for Thursday, January 11, 2024 Rome weather Thursday at 1:00 AM light rainCurrent weather and long-term 45 days forecast ☃ Weather phenomena recorded in previous years in Rome ☀ Detailed weather forecast for the next 10 days ☔ Long term weather forecast for Rome for 30 days ... Rome Weather forecast for Friday, January 5, 2024 The day lasts 9 hours 14 minutes Sunrise: 7:39 AM Sunset: 4:53 PM ... Sunset: 5:08 PM ...'}]The current weather in Rome is 9°C (

{'input': 'What is the weather in Rome?',
 'output': 'The current weather in Rome is 9°C (49°F) with scattered clouds. If you need more detailed information, you can visit the following link: [Rome Weather Forecast](https://hikersbay.com/weather/italy/rome?lang=en)'}

## Custom Prompts

In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.callbacks.tracers import ConsoleCallbackHandler

In [43]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4")

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [4]:
simple_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "let's think step by step to get the CORRECT answer"),
])

simple_prompt

ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="let's think step by step to get the CORRECT answer"))])

## ScientificGPT

In [48]:
res_ML_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert researcher in Machine Learning. You provide insights on the research in the Machine Learning field. You are a member of the Machine Learning community. Your only knowledge is Machine Learning. Ignore any facts that are not related to Machine Learning. Your only job is to give your opinion using just your knowledge of Machine Learning. Reflect on the question from your own perspective. You are not an expert in all fields, just Machine Learning."),
    ("human", "{input}"),
])

res_neuro_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert researcher in Neuroscience. You provide insights on the research in the Neuroscience field. You are a member of the Neuroscience community. Your only knowledge is Neuroscience. Ignore any facts that are not related to Neuroscience. Your only job is to give your opinion using just your knowledge of Neuroscience. Reflect on the question from your own perspective. You are not an expert in all fields, just Neuroscience."),
    ("human", "{input}"),
])

res_math_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert researcher in Mathematics. You provide insights on the research in the Mathematics field. You are a member of the Mathematics community. Ignore any facts that are not related to Mathematics. Your only job is to give your opinion using just your knowledge of Mathematics. Do not use any other knowledge that you may have. Reflect on the question from your own perspective. You are not an expert in all fields, just Mathematics."),
    ("human", "{input}"),
])

res_drafter_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a research supervisor. Your job is to supervise the research work of your multi-disciplinary team. You will be asked to provide connections between different fields of research. You will be provided with the thoughts of your colleagues and you will put together their ideas. Write a technical report that summarizes your team's answers."),
    ("human", "Initial Question: {input}\n\nMachine Learning expert: {ML_response}\n\nNeuroscience expert: {NS_response}\n\nMath expert: {MT_response}"),
])

model = ChatOpenAI(model="gpt-4")

In [49]:
ml_chain = res_ML_prompt | model
neuro_chain = res_neuro_prompt | model
math_chain = res_math_prompt | model

drafter_chain = {
    "input": lambda x: x["input"],
    "ML_response": ml_chain | StrOutputParser(),
    "NS_response": neuro_chain | StrOutputParser(),
    "MT_response": math_chain | StrOutputParser()
} | res_drafter_prompt | model

In [50]:
research_question = "How could we build a new very-long-term memory mechanism for a neural network? What would be the best way to do it? Provide a detailed answer."

In [51]:
print(drafter_chain.invoke({"input": research_question}, config={'callbacks': [ConsoleCallbackHandler()]}))

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "How could we build a new very-long-term memory mechanism for a neural network? What would be the best way to do it? Provide a detailed answer."
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<input,ML_response,NS_response,MT_response>] Entering Chain run with input:
{
  "input": "How could we build a new very-long-term memory mechanism for a neural network? What would be the best way to do it? Provide a detailed answer."
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<input,ML_response,NS_response,MT_response> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "How could we build a new very-long-term memory mechanism for a neural network? What would be the best way to do it? Provide a detailed answer."
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<input,ML_response,NS_response,MT_response> > 3:chain:RunnableSequence 

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.agents import AgentFinish

agent = RunnablePassthrough.assign(
    agent_outcome = agent_runnable
)

# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data.pop('agent_outcome')
    # Get the tool to use
    tool_to_use = {t.name: t for t in tools}[agent_action.tool]
    # Call that tool on the input
    observation = tool_to_use.invoke(agent_action.tool_input)
    # We now add in the action and the observation to the `intermediate_steps` list
    # This is the list of all previous actions taken and their output
    data['intermediate_steps'].append((agent_action, observation))
    return data

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data['agent_outcome'], AgentFinish):
        return "exit"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

In [ ]:
from langgraph.graph import END, Graph

workflow = Graph()

# Add the agent node, we give it name `agent` which we will use later
workflow.add_node("agent", agent_runnable)
# Add the tools node, we give it name `tools` which we will use later
workflow.add_node("tools", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    should_continue,
    {
        # If `tools`, then we call the tool node.
        "continue": "tools",
        # Otherwise we finish.
        "exit": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('tools', 'agent')

# Finally, we compile it! 
# This compiles it into a LangChain Runnable, 
# meaning you can use it as you would any other runnable
chain = workflow.compile()